In [14]:
import requests as req
from bs4 import BeautifulSoup as bs
import re
def sub_scraper(url, var):
    r = req.get(url)
    print(r.status_code)
    soup = bs(r.text, 'lxml')
    script_divs = soup.find_all('script', {'type': 'text/javascript'})
    res = 0
    for i in range(len(script_divs)):
#         print(i)
#         print(script_divs[i])
        if "CSV" in str(script_divs[i]):
            if var == 'count':
                res = script_divs[i]
            elif var == 'total':
                res = script_divs[i + 1]
            elif var == 'views':
                res = script_divs[i + 2]
            elif var == 'views_tot':
                res = script_divs[i + 3]
            break
#     print(res)
    lst = str(res).split('+')
    lst = [test.strip() for test in lst]
    lst = [test.replace('\\n"', '').replace('"', '') for test in lst]
    return lst

In [15]:
leafy = 'https://web.archive.org/web/20161218062757/https://socialblade.com/youtube/user/leafyishere/monthly'
sub_scraper(leafy, 'count')

200


["<script type=text/javascript>\t\t\n\tg = new Dygraph(\n\n\t// containing div\n\tdocument.getElementById('DailySubscribers'),\n\t// CSV or path to a CSV file.\n\tDate,Daily Subs",
 '2012-01-15,5',
 '2012-01-16,4',
 '2012-01-17,5',
 '2012-01-18,7',
 '2012-01-19,4',
 '2012-01-20,11',
 '2012-01-21,9',
 '2012-01-22,2',
 '2012-01-23,3',
 '2012-01-24,6',
 '2012-01-25,8',
 '2012-01-26,1',
 '2012-01-27,6',
 '2012-01-28,8',
 '2012-01-29,6',
 '2012-01-30,-1',
 '2012-01-31,11',
 '2012-02-01,4',
 '2012-02-02,14',
 '2012-02-03,2',
 '2012-02-04,22',
 '2012-02-05,4',
 '2012-02-06,16',
 '2012-02-07,10',
 '2012-02-08,13',
 '2012-02-09,17',
 '2012-02-10,12',
 '2012-02-11,31',
 '2012-02-12,14',
 '2012-02-13,8',
 '2012-02-14,23',
 '2012-02-15,10',
 '2012-02-16,10',
 '2012-02-17,8',
 '2012-02-18,34',
 '2012-02-19,16',
 '2012-02-20,13',
 '2012-02-21,24',
 '2012-02-22,7',
 '2012-02-23,10',
 '2012-02-24,7',
 '2012-02-25,29',
 '2012-02-26,11',
 '2012-02-27,37',
 '2012-02-28,4',
 '2012-02-29,17',
 '2012-03-01,

In [16]:
import pandas as pd
def to_df(url, name, var):
    lst = sub_scraper(url, var)
    print(len(lst))
    lst = lst[1:len(lst) - 1]
    df = pd.DataFrame()
    df['Date'] = [x.split(',')[0] for x in lst]
    df['Subs'] = [x.split(',')[1] for x in lst]
    df['Name'] = name
    return df

In [17]:
to_df(leafy, 'LeafyIsHere', 'count')

200
1799


,Date,Subs,Name
0,2012-01-15,5,LeafyIsHere
1,2012-01-16,4,LeafyIsHere
2,2012-01-17,5,LeafyIsHere
3,2012-01-18,7,LeafyIsHere
4,2012-01-19,4,LeafyIsHere
...,...,...,...
1792,2016-12-12,-2823,LeafyIsHere
1793,2016-12-13,-1764,LeafyIsHere
1794,2016-12-14,56,LeafyIsHere
1795,2016-12-15,-1842,LeafyIsHere


In [18]:
from datetime import date
from dateutil.relativedelta import relativedelta
def checkmonth(check, year, month, day):
    target = date(year, month, day)
    check = date.fromisoformat(check)
    bounds = [target + relativedelta(months=-1), target + relativedelta(months=+1)]
    if check >= bounds[0] and check <= bounds[1]:
        return True
    else: 
        return False
    
def filterdate(date_str, df):
    target = date.fromisoformat(date_str)
    month = target.month
    day   = target.day
    year = target.year
    return df[df['Date'].apply(checkmonth, args=(year, month, day))]

In [19]:
filterdate('2016-09-13', to_df(leafy, 'LeafyIsHere', 'count'))

200
1799


,Date,Subs,Name
1671,2016-08-13,16675,LeafyIsHere
1672,2016-08-14,19552,LeafyIsHere
1673,2016-08-15,13991,LeafyIsHere
1674,2016-08-16,12523,LeafyIsHere
1675,2016-08-17,11758,LeafyIsHere
...,...,...,...
1728,2016-10-09,241,LeafyIsHere
1729,2016-10-10,-2629,LeafyIsHere
1730,2016-10-11,-1712,LeafyIsHere
1731,2016-10-12,1395,LeafyIsHere


In [20]:
dates = """2015-12-13
2016-02-02
2016-05-05
2016-08-12
2016-09-13
2017-02-06
2017-10-03""".splitlines()
dates

['2015-12-13',
 '2016-02-02',
 '2016-05-05',
 '2016-08-12',
 '2016-09-13',
 '2017-02-06',
 '2017-10-03']

In [21]:
names = """Jinx
Fine Bros
Keemstar
HowToPRANKItUp
LeafyIsHere
Tana Mongeau
Ricegum""".splitlines()
names

['Jinx',
 'Fine Bros',
 'Keemstar',
 'HowToPRANKItUp',
 'LeafyIsHere',
 'Tana Mongeau',
 'Ricegum']

In [22]:
url_dict = {}
url_dict['Jinx'] = "https://web.archive.org/web/20160628012321/https://socialblade.com/youtube/user/starsosilly/monthly"
url_dict['Fine Bros'] = 'https://web.archive.org/web/20160826103708/https://socialblade.com/youtube/user/thefinebros/monthly'
url_dict['Keemstar'] = 'https://web.archive.org/web/20160716024604/https://socialblade.com/youtube/user/newdramaalert/monthly'
url_dict['HowToPRANKItUp'] = 'https://web.archive.org/web/20190606174203/https://socialblade.com/youtube/user/howtoprankitup/monthly'
url_dict['LeafyIsHere'] = 'https://web.archive.org/web/20161218062757/https://socialblade.com/youtube/user/leafyishere/monthly'
url_dict['Tana Mongeau'] = 'https://web.archive.org/web/20190524102656/https://socialblade.com/youtube/c/tanamongeau/monthly'
url_dict['Ricegum'] = 'https://web.archive.org/web/20190822083857/https://socialblade.com/youtube/user/ricegum/monthly'

In [23]:
url_dict

{'Jinx': 'https://web.archive.org/web/20160628012321/https://socialblade.com/youtube/user/starsosilly/monthly',
 'Fine Bros': 'https://web.archive.org/web/20160826103708/https://socialblade.com/youtube/user/thefinebros/monthly',
 'Keemstar': 'https://web.archive.org/web/20160716024604/https://socialblade.com/youtube/user/newdramaalert/monthly',
 'HowToPRANKItUp': 'https://web.archive.org/web/20190606174203/https://socialblade.com/youtube/user/howtoprankitup/monthly',
 'LeafyIsHere': 'https://web.archive.org/web/20161218062757/https://socialblade.com/youtube/user/leafyishere/monthly',
 'Tana Mongeau': 'https://web.archive.org/web/20190524102656/https://socialblade.com/youtube/c/tanamongeau/monthly',
 'Ricegum': 'https://web.archive.org/web/20190822083857/https://socialblade.com/youtube/user/ricegum/monthly'}

In [24]:
idubbbz = to_df("https://web.archive.org/web/20180618204050/https://socialblade.com/youtube/user/idubbbztv/monthly", "iDubbbzTV", "count")
idubbbz['total'] = to_df("https://web.archive.org/web/20180618204050/https://socialblade.com/youtube/user/idubbbztv/monthly", "iDubbbzTV","total").Subs
idubbbz['views'] = to_df("https://web.archive.org/web/20180618204050/https://socialblade.com/youtube/user/idubbbztv/monthly", "iDubbbzTV","views").Subs
idubbbz

200
1094
200
1095
200
1094


,Date,Subs,Name,total,views
0,2015-04-22,103,iDubbbzTV,96096,33530
1,2015-04-23,112,iDubbbzTV,96199,34424
2,2015-04-24,149,iDubbbzTV,96311,75400
3,2015-04-25,176,iDubbbzTV,96460,0
4,2015-04-26,70,iDubbbzTV,96636,54294
...,...,...,...,...,...
1087,2018-06-12,3717,iDubbbzTV,6614446,833294
1088,2018-06-13,3467,iDubbbzTV,6618163,799777
1089,2018-06-14,3379,iDubbbzTV,6621630,730187
1090,2018-06-15,2977,iDubbbzTV,6625009,693577


In [25]:
dfs = []
for i in range(len(names)):
    dfs.append(filterdate(dates[i], to_df(url_dict[names[i]], names[i], 'count')))
dfs

200
531
200
1799
200
736
200
1094
200
1799
200
1094
200
1094


[           Date  Subs  Name
 303  2015-11-13  6127  Jinx
 304  2015-11-14  4669  Jinx
 305  2015-11-15  4665  Jinx
 306  2015-11-16  3468  Jinx
 307  2015-11-17  4316  Jinx
 ..          ...   ...   ...
 360  2016-01-09  3453  Jinx
 361  2016-01-10  2535  Jinx
 362  2016-01-11  2522  Jinx
 363  2016-01-12  2292  Jinx
 364  2016-01-13  1956  Jinx
 
 [62 rows x 3 columns],
             Date   Subs       Name
 1561  2016-01-02   9290  Fine Bros
 1562  2016-01-03  12023  Fine Bros
 1563  2016-01-04   8959  Fine Bros
 1564  2016-01-05   8368  Fine Bros
 1565  2016-01-06   7801  Fine Bros
 ...          ...    ...        ...
 1617  2016-02-27   -965  Fine Bros
 1618  2016-02-28    765  Fine Bros
 1619  2016-02-29   1037  Fine Bros
 1620  2016-03-01   2809  Fine Bros
 1621  2016-03-02   1322  Fine Bros
 
 [61 rows x 3 columns],
            Date   Subs      Name
 634  2016-04-05  23851  Keemstar
 635  2016-04-06  13927  Keemstar
 636  2016-04-07  12856  Keemstar
 637  2016-04-08  11543  Keemsta

In [26]:
# idubbbz.to_csv('idubbbz_growth.csv', index=False)

In [30]:
tot_dfs = []
for i in range(len(names)):
    tot_dfs.append(filterdate(dates[i], to_df(url_dict[names[i]], names[i], 'total')))
tot_dfs

200
532
200
1800
200
737
200
1095
200
1800
200
1095
200
1095


[           Date     Subs  Name
 303  2015-11-13  1260181  Jinx
 304  2015-11-14  1266308  Jinx
 305  2015-11-15  1270977  Jinx
 306  2015-11-16  1275642  Jinx
 307  2015-11-17  1279110  Jinx
 ..          ...      ...   ...
 360  2016-01-09  1454666  Jinx
 361  2016-01-10  1458119  Jinx
 362  2016-01-11  1460654  Jinx
 363  2016-01-12  1463176  Jinx
 364  2016-01-13  1465468  Jinx
 
 [62 rows x 3 columns],
             Date      Subs       Name
 1561  2016-01-02  13859381  Fine Bros
 1562  2016-01-03  13868671  Fine Bros
 1563  2016-01-04  13880694  Fine Bros
 1564  2016-01-05  13889653  Fine Bros
 1565  2016-01-06  13898021  Fine Bros
 ...          ...       ...        ...
 1617  2016-02-27  13589950  Fine Bros
 1618  2016-02-28  13588985  Fine Bros
 1619  2016-02-29  13589750  Fine Bros
 1620  2016-03-01  13590787  Fine Bros
 1621  2016-03-02  13593596  Fine Bros
 
 [61 rows x 3 columns],
            Date     Subs      Name
 634  2016-04-05  1248530  Keemstar
 635  2016-04-06  127238

In [35]:
for i in range(len(dfs)):
    dfs[i]['Total'] = tot_dfs[i].Subs
dfs

[           Date  Subs  Name    Total
 303  2015-11-13  6127  Jinx  1260181
 304  2015-11-14  4669  Jinx  1266308
 305  2015-11-15  4665  Jinx  1270977
 306  2015-11-16  3468  Jinx  1275642
 307  2015-11-17  4316  Jinx  1279110
 ..          ...   ...   ...      ...
 360  2016-01-09  3453  Jinx  1454666
 361  2016-01-10  2535  Jinx  1458119
 362  2016-01-11  2522  Jinx  1460654
 363  2016-01-12  2292  Jinx  1463176
 364  2016-01-13  1956  Jinx  1465468
 
 [62 rows x 4 columns],
             Date   Subs       Name     Total
 1561  2016-01-02   9290  Fine Bros  13859381
 1562  2016-01-03  12023  Fine Bros  13868671
 1563  2016-01-04   8959  Fine Bros  13880694
 1564  2016-01-05   8368  Fine Bros  13889653
 1565  2016-01-06   7801  Fine Bros  13898021
 ...          ...    ...        ...       ...
 1617  2016-02-27   -965  Fine Bros  13589950
 1618  2016-02-28    765  Fine Bros  13588985
 1619  2016-02-29   1037  Fine Bros  13589750
 1620  2016-03-01   2809  Fine Bros  13590787
 1621  2016-

In [36]:
dfs[4]

,Date,Subs,Name,Total
1671,2016-08-13,16675,LeafyIsHere,4457205
1672,2016-08-14,19552,LeafyIsHere,4473880
1673,2016-08-15,13991,LeafyIsHere,4493432
1674,2016-08-16,12523,LeafyIsHere,4507423
1675,2016-08-17,11758,LeafyIsHere,4519946
...,...,...,...,...
1728,2016-10-09,241,LeafyIsHere,4783164
1729,2016-10-10,-2629,LeafyIsHere,4783405
1730,2016-10-11,-1712,LeafyIsHere,4780776
1731,2016-10-12,1395,LeafyIsHere,4779064


In [37]:
pd.concat(dfs).to_csv('contentcop_v2.csv', index=False)

In [38]:
url_dict.keys()

dict_keys(['Jinx', 'Fine Bros', 'Keemstar', 'HowToPRANKItUp', 'LeafyIsHere', 'Tana Mongeau', 'Ricegum'])

In [39]:
new = {}
new['Keemstar'] = 'https://web.archive.org/web/20190524110449/https://socialblade.com/youtube/user/newdramaalert/monthly'
new['LeafyIsHere'] = 'https://web.archive.org/web/20190830030622/https://socialblade.com/youtube/user/leafyishere/monthly'

In [40]:
cc_view_dfs = []
for key in ((url_dict.keys())):
    if key == 'LeafyIsHere' or key == 'Keemstar':
        temp = to_df(url_dict[key], key, "views")
        temp2 = to_df(new[key], key, "views")
        cc_view_dfs.append(pd.concat([temp, temp2]).drop_duplicates())
    else:
        cc_view_dfs.append(to_df(url_dict[key], key, "views"))
cc_view_dfs

200
531
200
1799
200
736
200
1094
200
1094
200
1799
200
1094
200
1094
200
1094


[           Date    Subs  Name
 0    2014-12-08     132  Jinx
 1    2014-12-09      93  Jinx
 2    2014-12-10      77  Jinx
 3    2014-12-11      95  Jinx
 4    2014-12-12     142  Jinx
 ..          ...     ...   ...
 524  2016-06-21  212971  Jinx
 525  2016-06-22  212555  Jinx
 526  2016-06-23  204525  Jinx
 527  2016-06-24  200117  Jinx
 528  2016-06-25  230600  Jinx
 
 [529 rows x 3 columns],
             Date     Subs       Name
 0     2011-09-24   301780  Fine Bros
 1     2011-09-25   612485  Fine Bros
 2     2011-09-26   677058  Fine Bros
 3     2011-09-27   666627  Fine Bros
 4     2011-09-28   367927  Fine Bros
 ...          ...      ...        ...
 1792  2016-08-20  2466961  Fine Bros
 1793  2016-08-21  2169020  Fine Bros
 1794  2016-08-22  2828388  Fine Bros
 1795  2016-08-23  2586710  Fine Bros
 1796  2016-08-24  2548439  Fine Bros
 
 [1797 rows x 3 columns],
             Date     Subs      Name
 0     2014-06-17        0  Keemstar
 1     2014-06-18        0  Keemstar
 2    

In [43]:
for i in range(len(cc_dfs)):
    cc_dfs[i]['Views'] = cc_view_dfs[i].Subs
cc_dfs

[           Date  Subs  Name   Views
 0    2014-12-08     5  Jinx     132
 1    2014-12-09     1  Jinx      93
 2    2014-12-10     1  Jinx      77
 3    2014-12-11     1  Jinx      95
 4    2014-12-12     2  Jinx     142
 ..          ...   ...   ...     ...
 524  2016-06-21  -193  Jinx  212971
 525  2016-06-22   539  Jinx  212555
 526  2016-06-23  -446  Jinx  204525
 527  2016-06-24  -334  Jinx  200117
 528  2016-06-25   346  Jinx  230600
 
 [529 rows x 4 columns],
             Date  Subs       Name    Views
 0     2011-09-24  1995  Fine Bros   301780
 1     2011-09-25  1521  Fine Bros   612485
 2     2011-09-26  2973  Fine Bros   677058
 3     2011-09-27  1807  Fine Bros   666627
 4     2011-09-28  1118  Fine Bros   367927
 ...          ...   ...        ...      ...
 1792  2016-08-20  3277  Fine Bros  2466961
 1793  2016-08-21  4214  Fine Bros  2169020
 1794  2016-08-22  2957  Fine Bros  2828388
 1795  2016-08-23  2088  Fine Bros  2586710
 1796  2016-08-24  2085  Fine Bros  2548439
 

In [44]:
pd.concat(cc_dfs).to_csv('contentcop_all_v2.csv', index=False)